In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

# Import from your codebase
from CLAPWrapper import CLAPWrapper
from datasets.esc50 import ESC50
from datasets.tinysol import TinySOL

DATASET = TinySOL

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [2]:
print("\n" + "="*80)
print(f"Loading {DATASET.__name__} Dataset")
print("="*80)

root_path = "../data"
dataset = DATASET(root=root_path, download=False)
print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"   Classes: {len(dataset.classes)} categories")
print(f"   Sample classes: {dataset.classes[:5]}")

# Prepare text prompts
prompt = 'this is the sound of '
text_labels = [prompt + x for x in dataset.classes]
print(f"\n📝 Text prompts: {len(text_labels)} classes")


Loading TinySOL Dataset
Dataset già presente in ../data/TinySOL
Loading audio files


2913it [00:00, 17117.59it/s]

✅ Dataset loaded: 2913 samples
   Classes: 14 categories
   Sample classes: ['Accordion', 'Bass Tuba', 'Bassoon', 'Clarinet Bb', 'Contrabass']

📝 Text prompts: 14 classes


In [3]:
# ============================================================================
# STEP 3: Initialize Models
# ============================================================================
print("\n" + "="*80)
print("Initializing Models")
print("="*80)

# Residual config con pc_weights = 1.0 (identità)
residual_config = {
    'mode': 'attention',
    'n_components_ratio': 1.0,
    'reweight_factor': 0.0,
    'target_layers': [0, 1, 2, 3],  # Layers dove applicare reweighting
}

print("\n🔧 Loading CLAP Standard...", end='')
clap_standard = CLAPWrapper(
    version='2023',  # or '2022'
    use_cuda=torch.cuda.is_available(),
    type='classic'
)
clap_standard.clap.eval()
clap_standard.clap.audio_encoder.base.htsat.eval()
print("OK")

print("\n🔧 Loading ResiDualCLAP...")
clap_residual = CLAPWrapper(
    version='2023',
    use_cuda=torch.cuda.is_available(),
    type='residual',
    residual_config=residual_config
)
clap_residual.clap.eval()
clap_residual.clap.audio_encoder.base.htsat.eval()


Initializing Models

🔧 Loading CLAP Standard...OK

🔧 Loading ResiDualCLAP...

🔧 Setup ResiDual HTSAT
Modalità: ATTENTION
Target layers: [0, 1, 2, 3]
PCA components ratio: 1.0
Reweight factor: 0.0

✓ layer_0:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 4
  Total heads: 8
  Head dim: 24D → 24 PCs

✓ layer_1:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 8
  Total heads: 16
  Head dim: 24D → 24 PCs

✓ layer_2:
  Modalità: PER-HEAD reweighting
  Num blocks: 6
  Heads per block: 16
  Total heads: 96
  Head dim: 24D → 24 PCs

✓ layer_3:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 32
  Total heads: 64
  Head dim: 24D → 24 PCs




ResiDualHTSAT(
  (spectrogram_extractor): Spectrogram(
    (stft): STFT(
      (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
      (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
    )
  )
  (logmel_extractor): LogmelFilterBank()
  (spec_augmenter): SpecAugmentation(
    (time_dropper): DropStripes()
    (freq_dropper): DropStripes()
  )
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=96, input_resolution=(64, 64), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=96, input_resolution=(64, 64), num_heads=4, window_size=8, shift_size=0, mlp_ratio=4.0
          (norm1): LayerNorm((96,), eps=1e-05, eleme

In [4]:
PCA_SAMPLES = 200

# Prepare audio samples for PCA fitting
print(f"Collecting samples for PCA fitting ({PCA_SAMPLES} samples)...", end='')

# Create a simple dataloader wrapper per PCA fitting
class SimpleAudioDataset:
    def __init__(self, wrapper, esc50_dataset, max_samples=1000):
        self.wrapper = wrapper
        self.audio_paths = []
        for i in range(min(max_samples, len(esc50_dataset))):
            audio_path, _, _ = esc50_dataset[i]
            self.audio_paths.append(audio_path)
    
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, idx):
        audio_tensor = self.wrapper.load_audio_into_tensor(
            self.audio_paths[idx],
            self.wrapper.args.duration,
            resample=True
        )
        # ✅ Assicurati sia 1D
        if audio_tensor.dim() > 1:
            audio_tensor = audio_tensor.squeeze()
        
        return audio_tensor

# Create dataset and loader
pca_dataset = SimpleAudioDataset(clap_residual, dataset, max_samples=PCA_SAMPLES)
pca_loader = DataLoader(
    pca_dataset, 
    batch_size=25, 
    shuffle=False,
    num_workers=0,  # Start with 0 for debugging
    pin_memory=False
)
print("OK")

In [5]:
pca_dataset[0].shape

torch.Size([308700])

In [6]:
list(pca_loader)[1].shape

torch.Size([25, 308700])

In [7]:
len(dataset)

2913

In [8]:
print("\n" + "="*80)
print("Fitting PCA Components")
print("="*80)

# Fit PCA
print(f"Fitting PCA on {len(pca_dataset)} samples...")
variance_ratios = clap_residual.clap.audio_encoder.base.htsat.fit_spectral_layers(
    pca_loader,
    max_samples=PCA_SAMPLES
)

print("\n📊 PCA Variance Ratios:")
print("="*80)

# Gestione corretta della struttura gerarchica
for layer_name, layer_data in variance_ratios.items():
    print(f"\n{layer_name}:")
    
    # Controlla se è mode 'attention' (struttura annidata) o 'layer' (array diretto)
    if isinstance(layer_data, dict):
        # Mode ATTENTION: layer_data contiene blocchi
        for block_name, block_data in layer_data.items():
            print(f"  {block_name}:")
            
            # block_data contiene le teste
            for head_name, head_variance in block_data.items():
                # head_variance è l'array numpy con le variance ratios
                top3 = head_variance[:3]
                cumulative = head_variance[:3].sum()
                print(f"    {head_name}: top3={[f'{v:.3f}' for v in top3]}, cum={cumulative:.3f}")
    else:
        # Mode LAYER: layer_data è direttamente l'array di variance ratios
        top5 = layer_data[:5]
        cumulative = layer_data[:5].sum()
        print(f"  Top 5 components: {[f'{v:.4f}' for v in top5]}")
        print(f"  Cumulative variance (top 5): {cumulative:.4f}")


Fitting PCA Components
Fitting PCA on 200 samples...

📊 Fitting Spectral Layers
Modalità: ATTENTION
Target layers: [0, 1, 2, 3]
Max samples: 200

📦 Fase 1: Raccolta hidden states...


Raccolta: 100%|██████████| 8/8 [00:35<00:00,  4.49s/it, samples=200]



✓ Raccolti 200 samples totali

🔬 Fase 2: Calcolo PCA e inizializzazione pesi...


layer_0:

  Block 0:
    ✓ head_0: shape=torch.Size([819200, 24]), var=['0.879', '0.061', '0.051'], cum=0.991
    ✓ head_1: shape=torch.Size([819200, 24]), var=['0.782', '0.090', '0.076'], cum=0.948
    ✓ head_2: shape=torch.Size([819200, 24]), var=['0.883', '0.072', '0.035'], cum=0.989
    ✓ head_3: shape=torch.Size([819200, 24]), var=['0.836', '0.070', '0.037'], cum=0.943

  Block 1:
    ✓ head_0: shape=torch.Size([819200, 24]), var=['0.345', '0.237', '0.149'], cum=0.730
    ✓ head_1: shape=torch.Size([819200, 24]), var=['0.750', '0.115', '0.042'], cum=0.907
    ✓ head_2: shape=torch.Size([819200, 24]), var=['0.620', '0.139', '0.116'], cum=0.875
    ✓ head_3: shape=torch.Size([819200, 24]), var=['0.538', '0.225', '0.105'], cum=0.868

layer_1:

  Block 0:
    ✓ head_0: shape=torch.Size([204800, 24]), var=['0.434', '0.158', '0.113'], cum=0.706
    ✓ head_1: shape=torch.Size([204800, 24]), var=['0.241', '

In [9]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_standard.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 300  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_baseline, y_labels = [], []

for i in tqdm(range(test_size), desc="Baseline"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_standard.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_standard.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_baseline.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_baseline_array = np.concatenate(y_preds_baseline, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Baseline Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([14, 1024])

📊 Testing on 300 samples...


Baseline:   0%|          | 0/300 [00:00<?, ?it/s]


✅ Baseline Accuracy: 0.487 (48.7%)


In [10]:
baseline_acc

0.4866666666666667

In [11]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_residual.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 300  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_residual, y_labels = [], []

for i in tqdm(range(test_size), desc="residual"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_residual.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_residual.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_residual.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_residual_array = np.concatenate(y_preds_residual, axis=0)

residual_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_residual_array, axis=1)
)

print(f"\n✅ Residual Accuracy: {residual_acc:.3f} ({residual_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([14, 1024])

📊 Testing on 300 samples...


residual:   0%|          | 0/300 [00:00<?, ?it/s]


✅ Residual Accuracy: 0.477 (47.7%)


In [12]:
residual_acc

0.4766666666666667

In [13]:
y_preds_residual_array

array([[8.6708739e-04, 1.2584196e-06, 2.9712135e-04, ..., 6.7006997e-03,
        3.0785173e-04, 2.0462200e-04],
       [2.2296207e-02, 3.8817632e-07, 9.7235944e-04, ..., 1.1185975e-01,
        1.8533451e-02, 1.6015100e-03],
       [3.2923691e-04, 1.4916168e-07, 2.2600570e-03, ..., 2.1775109e-03,
        1.9798054e-04, 2.3463765e-05],
       ...,
       [4.1160925e-05, 5.4688858e-06, 2.3399117e-04, ..., 4.6371697e-03,
        5.7084661e-04, 5.0900719e-04],
       [8.4686953e-06, 2.0872583e-06, 8.4836545e-05, ..., 1.3730243e-04,
        2.6141252e-05, 4.6615824e-05],
       [2.4429901e-06, 1.2070299e-06, 1.6206970e-05, ..., 2.3242106e-05,
        5.3767344e-06, 7.7631994e-06]], shape=(300, 14), dtype=float32)

In [14]:
y_preds_baseline_array 

array([[2.0666220e-03, 2.7926972e-06, 6.2200730e-04, ..., 1.8173505e-02,
        9.1775961e-04, 6.3625700e-04],
       [2.5382778e-03, 8.6734516e-08, 2.0553814e-03, ..., 1.6275084e-02,
        1.5918226e-03, 1.3957666e-04],
       [3.2923743e-04, 1.4916178e-07, 2.2600584e-03, ..., 2.1775123e-03,
        1.9798067e-04, 2.3463779e-05],
       ...,
       [4.1160984e-05, 5.4688940e-06, 2.3399139e-04, ..., 4.6371715e-03,
        5.7084742e-04, 5.0900766e-04],
       [8.4687190e-06, 2.0872642e-06, 8.4836785e-05, ..., 1.3730268e-04,
        2.6141302e-05, 4.6615911e-05],
       [2.4429924e-06, 1.2070299e-06, 1.6206954e-05, ..., 2.3242106e-05,
        5.3767399e-06, 7.7632076e-06]], shape=(300, 14), dtype=float32)

In [28]:
# ============================================================================
# VERIFICA: Tutti gli Spectral Layers sono stati fittati?
# ============================================================================

print("\n" + "="*80)
print("🔍 Verifica Stato Fitting di TUTTI gli Spectral Layers")
print("="*80)

htsat = clap_residual.clap.audio_encoder.base.htsat
all_fitted = True

for layer_idx in residual_config['target_layers']:
    layer_name = f'layer_{layer_idx}'
    
    if layer_name in htsat.spectral_layers:
        block_spectral_layers = htsat.spectral_layers[layer_name]
        
        for block_idx, head_spectral_layers in enumerate(block_spectral_layers):
            for head_idx, spectral_layer in enumerate(head_spectral_layers):
                is_fitted = spectral_layer.is_fitted.item()
                
                if not is_fitted:
                    print(f"❌ {layer_name}, Block {block_idx}, Head {head_idx}: NOT FITTED!")
                    all_fitted = False
                else:
                    # Verifica che i pesi siano effettivamente 1.0
                    weights = spectral_layer.pc_weights.data
                    all_ones = torch.allclose(weights, torch.ones_like(weights), atol=1e-6)
                    
                    if not all_ones:
                        print(f"⚠️  {layer_name}, Block {block_idx}, Head {head_idx}: "
                              f"Fitted ma pesi NON sono 1.0!")
                        print(f"    Primi 5 pesi: {weights[:5].tolist()}")
                        all_fitted = False

if all_fitted:
    print("\n✅ TUTTI i layer sono fittati correttamente con pesi = 1.0")
else:
    print("\n❌ PROBLEMA: Alcuni layer non sono fittati correttamente!")


🔍 Verifica Stato Fitting di TUTTI gli Spectral Layers

✅ TUTTI i layer sono fittati correttamente con pesi = 1.0


In [11]:
audio_path, target, one_hot_target = dataset[0]

In [12]:
clap_residual.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.6079,  0.6576, -0.5101,  ...,  1.8574,  0.9557,  0.2268]])

In [13]:
clap_standard.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])

In [14]:
torch.allclose(x, y, atol=1e-5)

NameError: name 'x' is not defined

In [ ]:
abs(x - y).max()

tensor(1.5497e-06)

In [ ]:
abs(x - y).sum()

tensor(0.0003)

In [ ]:
len(x[0])

1024